In [1]:
import sys
import os 
from pathlib import Path

import pickle 
import numpy as np
import pandas as pd
sys.path.insert(0,str('/Users/ines/research/Lingpred'))
sys.path.insert(0,str(Path().absolute()/"Lingpred"))
import lingpred_new
from lingpred_new.encoding_analysis import self_predictability
    
import seaborn as sns
import matplotlib as plt
from matplotlib import pyplot as plt
sns.set_theme()

In [ ]:
dataset = 'Armani'
#dataset = 'Gwilliams'
#dataset = 'Goldstein'
data_path    = '/project/3018059.03/Lingpred/data/{}/self_predictability/'.format(dataset)
results_path = '/project/3018059.03/Lingpred/results/{}/self_predictability/'.format(dataset)

In [ ]:

# load dataframe with the surprisal & prediction values:
path = '/project/3018059.03/Lingpred/data/{}/Dataframe_WordSurprisal_Top10Words_All_Sessions.pkl'.format(dataset)
df   = pickle.load(open(path, 'rb'))
        
# make masks for correct and incorrect top-5 prediction
is_top_5      = df.is_Top_5.to_numpy()
is_not_top_5  = np.array([not x for x in is_top_5])

# make masks for correct and incorrect top-5 prediction
is_top_1      = df.is_Top_1.to_numpy()
is_not_top_1  = np.array([not x for x in is_top_1])

In [ ]:
# get dataframe to make the bigram mask:
df_words_all = pd.DataFrame()

for session in np.arange(1, 11):
    df_words = lingpred_new.io.get_words_onsets_offsets(None, 
                                                    dataset=dataset, 
                                                    subject=1, 
                                                    session=session, 
                                                    run=1)
    df_words_all = df_words_all.append(df_words)

bigram_mask = lingpred_new.utils.get_bigram_mask(df_words_all)

In [ ]:
if dataset== 'Armani':
    file_name  = 'X_y_matrices_Glove_GPT_arbitrary_session_1.pkl'
if dataset== 'Gwilliams':
    file_name  = 'X_y_matrices_GPT_Glove_arbitrary.pkl'
filepath = data_path + file_name
data     = pickle.load(open(filepath, 'rb'))
data.keys()

In [ ]:
bigram_mask_first_session = [x for x in bigram_mask if x < 8622]
len(bigram_mask_first_session)

In [ ]:
X = data['X_Glove']
y = data['y_Glove']
selfpred_GloVe_no_bigrams = self_predictability(X = X[bigram_mask_first_session],
                                                y = np.swapaxes(y[bigram_mask_first_session], 0, 1))

X = data['X_GPT']
y = data['y_GPT']
selfpred_GPT_no_bigrams = self_predictability(X = X[bigram_mask_first_session],
                                                y = np.swapaxes(y[bigram_mask_first_session], 0, 1))

X = data['X_arbitrary']
y = data['y_arbitrary']
selfpred_arbitrary_no_bigrams = self_predictability(X = X[bigram_mask_first_session],
                                                    y = np.swapaxes(y[bigram_mask_first_session], 0, 1))


# and save:
filename = 'bigrams_removed_Glove_GPT_Arbitrary_session_1.pkl'
filepath = results_path + filename

values = [selfpred_GloVe_no_bigrams, selfpred_GPT_no_bigrams, selfpred_arbitrary_no_bigrams]
keys   = ['corr_GloVe', 'corr_GPT', 'corr_Arbitrary']
results= dict(zip(keys, values))

f = open(filepath,"wb")
pickle.dump(results,f)
f.close()